# **Final Project: Arcade**
### **Submitted By:**
#### Kashaf Qureshi
#### Fatima Tareen
#### Fatima Zahra
#### Zubia Tanoli

In [ ]:

import random 
from colorama import Fore
import math
import heapq
def print_colored_text(text, color):
    """Function to print text in a specified color."""
    colors = {
        'red': Fore.RED,
        'green': Fore.GREEN,
        'yellow': Fore.YELLOW,
        'blue': Fore.BLUE,
        'magenta': Fore.MAGENTA,
        'cyan': Fore.CYAN,
        'white': Fore.WHITE,
    }
    return colors.get(color, Fore.WHITE) + text + Fore.RESET

def main_menu():
    print(print_colored_text("\t\t\t\t\t---< ARCADE >---\n\n", 'red'))
    print("Choose a game to play:")
    print("1. Tic-Tac-Toe")
    print("2. Connect Four")
    print("3. Maze Solver")
    print("4. Quit\n\n\n")
    choice = input(print_colored_text("Enter your choice (1-5): ", 'yellow'))
    return choice

# Tic-Tac-Toe Implementation
def ttt_rules():
    print(print_colored_text("""\n\n---< TIC-TAC-TOE RULES >---
        ➤ Objective: Get 3 of your marks (X or O) in a row (horizontally, vertically, or diagonally).
        How to Play:
        • Players take turns placing either an "X" or an "O" on the grid.
        • The first player to get 3 of their marks in a row wins.
        • If all spaces are filled and no one has 3 in a row, it's a draw.\n\n\n""", 'blue'))

def print_board(board):
    for i, row in enumerate(board):
        print(" | ".join(row))
        if i != 2: # Skip the last separator line
            print("--+---+---")

def check_winner(board):
    for row in board:  # Check rows
        if row[0] == row[1] == row[2] and row[0] != ' ':
            return row[0]
    for col in range(3):  # Check columns
        if board[0][col] == board[1][col] == board[2][col] and board[0][col] != ' ':
            return board[0][col]
    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != ' ':  # Check diagonal
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != ' ':  # Check reverse diagonal
        return board[0][2]
    return None  # No winner

def is_full(board):
    return all(cell != ' ' for row in board for cell in row)

def evaluate(board):
    winner = check_winner(board)
    if winner == 'O':  # AI win
        return 10
    elif winner == 'X':  # Human win
        return -10
    return 0  # Draw or incomplete

def minimax(board, depth, is_maximizing):
    score = evaluate(board)
    if score == 10 or score == -10 or is_full(board):  # Terminal state
        return score

    if is_maximizing:  # Maximizing player (AI)
        best = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'O'
                    best = max(best, minimax(board, depth + 1, False))
                    board[i][j] = ' '
        return best
    else:  # Minimizing player (Human)
        best = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'X'
                    best = min(best, minimax(board, depth + 1, True))
                    board[i][j] = ' '
        return best

def find_best_move(board):
    best_val = -math.inf
    best_move = (-1, -1)
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ':
                board[i][j] = 'O'
                move_val = minimax(board, 0, False)
                board[i][j] = ' '
                if move_val > best_val:
                    best_val = move_val
                    best_move = (i, j)
    return best_move

def play_tic_tac_toe():
    ttt_rules()
    print(print_colored_text("\n\t\t\t\t----< Welcome to Tic-Tac-Toe! >----\n\n", "red"))
    print(print_colored_text("Choose Game Mode:\n1. Single Player\n2. Multiplayer\n", 'green'))
    mode = input(print_colored_text("Enter 1 or 2: ", 'yellow'))

    board = [[' ' for _ in range(3)] for _ in range(3)]
    player_turn = 'X'

    while True:
        print_board(board)
        print(print_colored_text(f"Player {player_turn}'s turn!", 'blue'))

        if mode == '1' and player_turn == 'O':  # AI's turn in single-player mode
            print("AI's Move...")
            row, col = find_best_move(board)
            board[row][col] = 'O'
        else:  # Human player input
            try:
                row, col = map(int, input("Enter your move (row and column 0-2, space-separated): ").split())
                if board[row][col] != ' ':
                    print(print_colored_text("Cell is already occupied. Try again.", 'red'))
                    continue
                board[row][col] = player_turn
            except (ValueError, IndexError):
                print(print_colored_text("Invalid input. Enter row and column as numbers 0-2.", 'red'))
                continue

        winner = check_winner(board)
        if winner:
            print_board(board)
            print(print_colored_text(f"Player {winner} wins!", 'magenta'))
            return
        if is_full(board):
            print_board(board)
            print(print_colored_text("It's a draw!", 'yellow'))
            return

        # Switch turns
        player_turn = 'O' if player_turn == 'X' else 'X'



                                                       # Connect Four Implementation

def cf_rules():
    print(print_colored_text("""\n\n---< CONNECT FOUR RULES >---

        ➡ Objective: Be the first to get 4 of your colored discs in a row (horizontally, vertically, or diagonally).

        How to Play:

        • Players take turns dropping one disc at a time into a column.
        • The disc will fall to the lowest available spot in the column.
        • The first player to align 4 discs in a row wins.
        • If the grid is full and no one has 4 in a row, it's a draw.\n\n\n""", 'blue'))

def evaluate_cf(board, player, opponent):
    """Enhanced evaluation function to assign scores for patterns."""
    score = 0
    rows, cols = len(board), len(board[0])

    def score_window(window):
        """Assign a score to a window of 4 cells."""
        score = 0
        if window.count(player) == 4:
            score += 1000
        elif window.count(player) == 3 and window.count(" ") == 1:
            score += 5
        elif window.count(player) == 2 and window.count(" ") == 2:
            score += 2

        if window.count(opponent) == 3 and window.count(" ") == 1:
            score -= 4

        return score

    # Horizontal
    for r in range(rows):
        for c in range(cols - 3):
            window = board[r][c:c + 4]
            score += score_window(window)

    # Vertical
    for c in range(cols):
        for r in range(rows - 3):
            window = [board[r + i][c] for i in range(4)]
            score += score_window(window)

    # Positive diagonal
    for r in range(rows - 3):
        for c in range(cols - 3):
            window = [board[r + i][c + i] for i in range(4)]
            score += score_window(window)

    # Negative diagonal
    for r in range(3, rows):
        for c in range(cols - 3):
            window = [board[r - i][c + i] for i in range(4)]
            score += score_window(window)

    return score


def is_terminal_node_cf(board):
# Check if the board state is terminal.
    return evaluate_cf(board, 'X', 'O') in [1000, -1000] or all(board[0][c] != ' ' for c in range(len(board[0])))

def minimax_cf(board, depth, alpha, beta, maximizingPlayer):
# Minimax with Alpha-Beta Pruning for Connect Four.
    opponent = 'O' if maximizingPlayer else 'X'
    player = 'X' if maximizingPlayer else 'O'

    if depth == 0 or is_terminal_node_cf(board):
        return evaluate_cf(board, player, opponent), None

    valid_columns = [c for c in range(len(board[0])) if board[0][c] == ' ']
    if maximizingPlayer:
        value = -math.inf
        best_col = random.choice(valid_columns)
        for col in valid_columns:
            row = next(r for r in range(len(board)-1, -1, -1) if board[r][col] == ' ')
            board[row][col] = 'X'
            new_score, _ = minimax_cf(board, depth-1, alpha, beta, False)
            board[row][col] = ' '
            if new_score > value:
                value = new_score
                best_col = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value, best_col
    else:
        value = math.inf
        best_col = random.choice(valid_columns)
        for col in valid_columns:
            row = next(r for r in range(len(board)-1, -1, -1) if board[r][col] == ' ') #selects lowest empty row in the selected col
            board[row][col] = 'O'
            new_score, _ = minimax_cf(board, depth-1, alpha, beta, True) #evaluate score after move decreasing depth by 1
            board[row][col] = ' '  # undo last move to check for better options
            if new_score < value:
                value = new_score
                best_col = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return value, best_col

def play_connect_four():
    cf_rules()
    print(print_colored_text("\n\t\t\t\t----< Welcome to Connect Four! >----\n\n", "red"))
    print(print_colored_text("Choose Game Mode:\n1. Single Player\n2. Multiplayer\n", 'green'))
    mode = input(print_colored_text("Enter 1 or 2: ", 'yellow'))

    rows, cols = 6, 7
    board = [[" " for _ in range(cols)] for _ in range(rows)]
    player = "X"

    def print_board():
        for i, row in enumerate(board):
            print(" | ".join(row))
            if i != rows - 1:  # Skip the last separator line
                print("--+---+---+---+---+---+---")

    def check_winner():
        """Check if there is a winner."""
        # Horizontal check
        for r in range(rows):
            for c in range(cols - 3):
                if board[r][c] == board[r][c + 1] == board[r][c + 2] == board[r][c + 3] and board[r][c] != " ":
                    return board[r][c]

        # Vertical check
        for c in range(cols):
            for r in range(rows - 3):
                if board[r][c] == board[r + 1][c] == board[r + 2][c] == board[r + 3][c] and board[r][c] != " ":
                    return board[r][c]

        # Positive diagonal check
        for r in range(rows - 3):
            for c in range(cols - 3):
                if board[r][c] == board[r + 1][c + 1] == board[r + 2][c + 2] == board[r + 3][c + 3] and board[r][c] != " ":
                    return board[r][c]

        # Negative diagonal check
        for r in range(3, rows):
            for c in range(cols - 3):
                if board[r][c] == board[r - 1][c + 1] == board[r - 2][c + 2] == board[r - 3][c + 3] and board[r][c] != " ":
                    return board[r][c]

        return None

    while True:
        print_board()
        if mode == '1' and player == "O":
            print("AI's Turn...")
            _, col = minimax_cf(board, 4, -math.inf, math.inf, True) #return col to ai for best move
        else:
            try:
                col = int(input(print_colored_text(f"Player {player}, choose a column (0-{cols - 1}): ", 'blue')))
                if col < 0 or col >= cols or board[0][col] != " ":
                    print(print_colored_text("Invalid column or column is full. Try again.", 'red'))
                    continue
            except ValueError:
                print(print_colored_text("Invalid input. Enter a number between 0 and 6.", 'red'))
                continue

        # Drop the piece in the selected column
        for row in range(rows - 1, -1, -1):
            if board[row][col] == " ":
                board[row][col] = player
                break

        # Check for a winner
        winner = check_winner()
        if winner:
            print_board()
            print(print_colored_text(f"Player {winner} wins!", 'magenta'))
            return

        # Check for a draw
        if all(board[0][c] != " " for c in range(cols)):
            print_board()
            print(print_colored_text("It's a draw!", 'yellow'))
            return

        # Switch players
        player = "O" if player == "X" else "X"
	




                                                         # Maze Solver Implementation
maze = [
        [1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1],
        [1, 0, 1, 0, 1, 1, 1],
        [1, 0, 1, 0, 1, 1, 1],
        [1, 0, 1, 0, 1, 1, 1],
        [1, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]
    ]
    

start = (1, 0)
end = (5, 6)

directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

def ucs(maze, start, end):
    rows, cols = len(maze), len(maze[0])
    # Priority queue to expand nodes with the least cost
    queue = [(0, start)]  # (cost, position)
    parent = {start: None}
    cost_so_far = {start: 0}  # Stores the cost to reach each position

    while queue:
        current_cost, current = heapq.heappop(queue)

        if current == end:
            path = []
            while current:
                path.append(current)
                current = parent[current]
            return path[::-1], current_cost  # Return the path and the total cost

        for direction in directions:
            next_row, next_col = current[0] + direction[0], current[1] + direction[1]
            if 0 <= next_row < rows and 0 <= next_col < cols and maze[next_row][next_col] == 0:
                next_position = (next_row, next_col)
                new_cost = current_cost + 1  # Every move has a cost of 1

                if next_position not in cost_so_far or new_cost < cost_so_far[next_position]:
                    cost_so_far[next_position] = new_cost
                    parent[next_position] = current
                    heapq.heappush(queue, (new_cost, next_position))
                    
    return [], 0  # If no path is found, return empty path and cost 0

def display_maze(maze, path=[], show_cost=False):
    for row in range(len(maze)):
        for col in range(len(maze[row])):
            if (row, col) == start:
                print("S", end=" ")  # Start position
            elif (row, col) == end:
                print("E", end=" ")  # End position
            elif (row, col) in path:
                print(".", end=" ")  # Path
            elif maze[row][col] == 1:
                print("#", end=" ")  # Wall
            else:
                print(" ", end=" ")  # Empty space
        print()
    if show_cost:
        print(f"Path Cost: {len(path) - 1} moves")  # Each move costs 1

def maze_game():
    print(print_colored_text("\n\t\t\t\t----< Welcome to the Maze Solver! >----\n", "red"))
    print("\nMaze Layout:")
    display_maze(maze)

    print(print_colored_text("\nChoose Game Mode:","green"))
    print(print_colored_text("1. Solve the Maze (AI)", "green"))
    print(print_colored_text("2. Solve the Maze (Player)", "green"))

    choice = input("Enter 1 or 2: ")

    if choice == '1':
        print(print_colored_text("\nAI is solving the maze...\n", "blue"))
        path, total_cost = ucs(maze, start, end)
        if path:
            display_maze(maze, path, show_cost=True)
            print(print_colored_text(f"Path found with total moves: {total_cost}", "blue"))
        else:
            print("No path found.")
    elif choice == '2':
        print(print_colored_text("\nSolve the Maze yourself (Player)!", "blue"))
        player_position = list(start)
        total_cost = 0

        while True:
            display_maze(maze)
            print(print_colored_text(f"Current moves: {total_cost}", "blue"))
            move = input("Enter your move (w=up, s=down, a=left, d=right): ").lower()
            if move == 'w':
                new_position = [player_position[0] - 1, player_position[1]]
            elif move == 's':
                new_position = [player_position[0] + 1, player_position[1]]
            elif move == 'a':
                new_position = [player_position[0], player_position[1] - 1]
            elif move == 'd':
                new_position = [player_position[0], player_position[1] + 1]
            else:
                print("Invalid move. Try again.")
                continue

            if (0 <= new_position[0] < len(maze)) and (0 <= new_position[1] < len(maze[0])) and maze[new_position[0]][new_position[1]] == 0:
                player_position = new_position
                total_cost += 1  # Each move costs 1
                if player_position == list(end):
                    print(print_colored_text("Congratulations, you've solved the maze!", "yellow"))
                    print(print_colored_text(f"Total moves: {total_cost}", "blue"))
                    break
            else:
                print("Invalid move. You hit a wall!")


# Main Program
def arcade_game():
    while True:
        choice = main_menu()
        if choice == '1':
            play_tic_tac_toe()
        elif choice == '2':
            play_connect_four()
        elif choice == '3':
            maze_game()
        elif choice == '4':
            print(print_colored_text("Thanks for playing! Goodbye!", 'cyan'))
            break
        else:
            print(print_colored_text("Invalid choice. Please try again.", 'red'))

if __name__ == "__main__":
    arcade_game()
	


					---< ARCADE >---


Choose a game to play:
1. Tic-Tac-Toe
2. Connect Four
3. Maze Solver
4. Quit





Enter your choice (1-5):  3



				----< Welcome to the Maze Solver! >----


Maze Layout:
# # # # # # # 
S         # # 
#   #   # # # 
#   #   # # # 
#   #   # # # 
#   #       E 
#         # # 
# # # # # # # 

Choose Game Mode:
1. Solve the Maze (AI)
2. Solve the Maze (Player)


Enter 1 or 2:  2



Solve the Maze yourself (Player)!
# # # # # # # 
S         # # 
#   #   # # # 
#   #   # # # 
#   #   # # # 
#   #       E 
#         # # 
# # # # # # # 
Current moves: 0


...

Invalid move. You hit a wall!
# # # # # # # 
S         # # 
#   #   # # # 
#   #   # # # 
#   #   # # # 
#   #       E 
#         # # 
# # # # # # # 
Current moves: 0


Enter your move (w=up, s=down, a=left, d=right):  d


# # # # # # # 
S         # # 
#   #   # # # 
#   #   # # # 
#   #   # # # 
#   #       E 
#         # # 
# # # # # # # 
Current moves: 1


Enter your move (w=up, s=down, a=left, d=right):  d


# # # # # # # 
S         # # 
#   #   # # # 
#   #   # # # 
#   #   # # # 
#   #       E 
#         # # 
# # # # # # # 
Current moves: 2
